In [63]:
import investpy

In [64]:
# Data manipulation modules
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import yfinance as yf

# Data visualization modules
import matplotlib.pyplot as plt

# Machine Learning modules
from sklearn.svm import SVC

# Cross-validation modules 
from sklearn.model_selection import TimeSeriesSplit

# Grid search module
from sklearn.model_selection import GridSearchCV

# Evaluation metrics modules
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [65]:
def get_train_test_data(cripto_series, test_percentage):
    '''This functions creates the training and test data for the stock time series'''
    time_series_length = len(cripto_series)
    test_series = cripto_series.iloc[int(time_series_length*(1-test_percentage)):time_series_length]
    train_series = cripto_series.iloc[:-int(time_series_length*test_percentage)]
    
    return(test_series, train_series)

def get_rate_of_change(cripto_series, variable, n_days):
    '''This function computes several values for the rate of change and add them into a dataset'''
    for day in n_days: 
        variable_name = f"{day}_days_of_change"
        cripto_series[variable_name] = (cripto_series[variable] - cripto_series[variable].shift(day)) / cripto_series[variable].shift(day)
        
def get_moving_average(cripto_series, variable, n_days):
    '''This function computes several values for the moving average and add them into a dataset'''
    for day in n_days:
        variable_name = f"{day}_day_moving_average"
        cripto_series[variable_name] = cripto_series[variable].rolling(day).mean()
        
def get_ratio(cripto_series, variable_1, variable_2):
    '''This function computes the ratio between two variables and add it into a dataset'''
    variable_name = f"{variable_1}_{variable_2}_ratio"
    cripto_series[variable_name] = cripto_series[variable_1] / cripto_series[variable_2]
    
def get_difference(cripto_series, variable_1, variable_2):
    '''This function computes the difference between the values of two variables and add it into a dataset'''
    variable_name = f"{variable_1}_{variable_2}_difference"
    cripto_series[variable_name] = cripto_series[variable_1] - cripto_series[variable_2] 
    
def train_and_predict(model, train_predictors, train_target, test_predictors):
    '''This function fits a model and then make predictions on it for a given amount of data'''
    model.fit(train_predictors, train_target)
    preds = model.predict(test_predictors)
    preds = pd.Series(preds, index=test_series_2.index)
    return preds

In [66]:
stock_series = investpy.stocks.get_stock_historical_data('PETR4', 'brazil', from_date = '01/01/2002', to_date='01/08/2022')

In [10]:
stock_series.head()

,Open,High,Low,Close,Volume,Currency
Date,,,,,,
2002-01-02,4.73,4.79,4.72,4.73,23979552,BRL
2002-01-03,4.75,4.80,4.74,4.76,39124800,BRL
2002-01-04,4.76,4.76,4.67,4.73,21548832,BRL
2002-01-07,4.71,4.87,4.71,4.83,40547104,BRL
2002-01-08,4.83,4.83,4.77,4.80,34469888,BRL


In [11]:
test_series, train_series = get_train_test_data(stock_series, test_percentage=0.1)

## Applying the new variables

In [12]:
train_series['target']  = np.array(train_series['Close'].shift(-1) > train_series['Close']).astype(int) 

In [13]:
get_rate_of_change(train_series, 'Close', [3, 5, 7, 15])

In [14]:
get_difference(train_series, 'Close', 'Open')
get_difference(train_series, 'High', 'Low')

In [39]:
train_series['low_dif'] = (train_series['Low'] - train_series['Low'].shift(1))
train_series['high_dif'] = train_series['High'] - train_series['High'].shift(1)

In [16]:
get_ratio(train_series, 'Open', 'Close')
get_ratio(train_series, 'High', 'Close')
get_ratio(train_series, 'Low', 'Close')

In [17]:
train_series.drop('Currency', axis=1, inplace=True)

In [43]:
train_series.dropna(axis=0, inplace=True)

In [44]:
test_series['target']  = np.array(test_series['Close'].shift(-1) > test_series['Close']).astype(int) 

In [45]:
get_rate_of_change(test_series, 'Close', [3, 5, 7, 15])

In [46]:
get_difference(test_series, 'Close', 'Open')
get_difference(test_series, 'High', 'Low')

In [47]:
test_series['low_dif'] = (test_series['Low'] - test_series['Low'].shift(1))
test_series['high_dif'] = test_series['High'] - test_series['High'].shift(1)

In [48]:
get_ratio(test_series, 'Open', 'Close')
get_ratio(test_series, 'High', 'Close')
get_ratio(test_series, 'Low', 'Close')

In [51]:
test_series.dropna(axis=0, inplace=True)

In [52]:
print(train_series.columns)
print(test_series.columns)

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'target', '3_days_of_change',
       '5_days_of_change', '7_days_of_change', '15_days_of_change',
       'Close_Open_difference', 'High_Low_difference', 'Open_Close_ratio',
       'High_Close_ratio', 'Low_Close_ratio', 'low_dif', 'high_dif'],
      dtype='object')
Index(['Open', 'High', 'Low', 'Close', 'Volume', 'target', '3_days_of_change',
       '5_days_of_change', '7_days_of_change', '15_days_of_change',
       'Close_Open_difference', 'High_Low_difference', 'low_dif', 'high_dif',
       'Open_Close_ratio', 'High_Close_ratio', 'Low_Close_ratio'],
      dtype='object')


In [53]:
predictors = ['Open', 'High', 'Low', 'Close', 'Volume',
       '3_days_of_change', '5_days_of_change', '7_days_of_change',
       '15_days_of_change', 'Close_Open_difference', 'High_Low_difference',
       'low_dif', 'high_dif', 'Open_Close_ratio', 'High_Close_ratio',
       'Low_Close_ratio']

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit()

from sklearn.model_selection import GridSearchCV
param_grid = [
    {'kernel': ['poly'], 'degree': [3, 4, 5], 'C': [0.01, 0.1, 1]}
]

svc_class = SVC()

grid_search = GridSearchCV(svc_class, param_grid, cv=tscv,
                           scoring='precision')

grid_search.fit(train_series[predictors], train_series['target'])

In [ ]:
grid_search.best_params_

In [54]:
model = SVC(kernel='poly', C=0.01, random_state=True)

In [55]:
model.fit(train_series[predictors], train_series['target'])

SVC(C=0.01, kernel='poly', random_state=True)

In [56]:
preds = model.predict(test_series[predictors])
preds = pd.Series(preds, index=test_series.index)

In [57]:
preds

Date
2020-08-25    1
2020-08-26    1
2020-08-27    1
2020-08-28    1
2020-08-31    1
             ..
2022-07-26    1
2022-07-27    1
2022-07-28    1
2022-07-29    1
2022-08-01    1
Length: 480, dtype: int32

In [58]:
precision_score(test_series['target'], preds)

0.51875

In [59]:
recall_score(test_series['target'], preds)

1.0

In [60]:
accuracy_score(test_series['target'], preds)

0.51875

In [61]:
confusion_matrix(test_series['target'], preds)

array([[  0, 231],
       [  0, 249]], dtype=int64)

In [62]:
preds

Date
2020-08-25    1
2020-08-26    1
2020-08-27    1
2020-08-28    1
2020-08-31    1
             ..
2022-07-26    1
2022-07-27    1
2022-07-28    1
2022-07-29    1
2022-08-01    1
Length: 480, dtype: int32